# Comparaison Multi-Modèles : SDXL Turbo, Qwen2-VL, Z-Image

**Module :** 03-Images-Orchestration
**Niveau :** Intermédiaire
**Durée estimée :** 45 minutes

## Objectifs
Ce notebook vous guide dans l'orchestration de multiples moteurs de génération d'images. Vous allez :
1.  Configurer un client unifié pour Forge (SDXL Turbo) et ComfyUI (Z-Image).
2.  Lancer une génération comparative sur le même prompt.
3.  Analyser les différences de style, de qualité et de vitesse.

## Architecture
- **SDXL Turbo (via Forge)** : Modèle ultra-rapide (1 step), idéal pour le prototypage.
- **Z-Image (via ComfyUI)** : Modèle haute qualité basé sur Lumina-2, idéal pour le rendu final.


In [ ]:
# 1. Setup Environnement
import os
import requests
import time
import json
import base64
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv
import matplotlib.pyplot as plt

# Chargement Token Auth
load_dotenv("../00-GenAI-Environment/.env")
COMFYUI_TOKEN = os.getenv("COMFYUI_AUTH_TOKEN")

# Configuration Services
SERVICES = {
    "forge": {
        "url": "http://localhost:7865",
        "type": "sd_webui"
    },
    "comfy": {
        "url": "http://localhost:8188",
        "type": "comfyui",
        "token": COMFYUI_TOKEN
    }
}

print("✅ Configuration chargée")

In [ ]:
# 2. Définition des Clients API

def generate_forge(prompt, seed=-1):
    payload = {
        "prompt": prompt,
        "steps": 1,
        "width": 512,
        "height": 512,
        "cfg_scale": 1.0,
        "sampler_name": "Euler a",
        "seed": seed
    }
    try:
        start = time.time()
        resp = requests.post(f"{SERVICES['forge']['url']}/sdapi/v1/txt2img", json=payload, timeout=10)
        duration = time.time() - start
        
        if resp.status_code == 200:
            img_data = base64.b64decode(resp.json()['images'][0])
            return Image.open(BytesIO(img_data)), duration
    except Exception as e:
        print(f"Forge Error: {e}")
    return None, 0

def generate_z_image(prompt, seed=42):
    # Workflow Z-Image (simplifié pour lisibilité)
    workflow = {
        "3": {"class_type": "KSampler", "inputs": {"seed": seed, "steps": 20, "cfg": 3.0, "sampler_name": "euler", "scheduler": "normal", "denoise": 1.0, "model": ["4", 0], "positive": ["6", 0], "negative": ["7", 0], "latent_image": ["5", 0]}},
        "4": {"class_type": "UnetLoaderGGUF", "inputs": {"unet_name": "z_image_turbo-Q5_K_M.gguf"}},
        "5": {"class_type": "EmptyLatentImage", "inputs": {"width": 1024, "height": 1024, "batch_size": 1}},
        "6": {"class_type": "CLIPTextEncode", "inputs": {"text": prompt, "clip": ["10", 0]}},
        "7": {"class_type": "CLIPTextEncode", "inputs": {"text": "low quality", "clip": ["10", 0]}},
        "8": {"class_type": "VAEDecode", "inputs": {"samples": ["12", 0], "vae": ["11", 0]}},
        "9": {"class_type": "SaveImage", "inputs": {"filename_prefix": "Comp_Z", "images": ["8", 0]}},
        "10": {"class_type": "CLIPLoaderGGUF", "inputs": {"clip_name": "gemma-3-4b-it-Q4_K_M.gguf", "type": "lumina2"}},
        "11": {"class_type": "VAELoader", "inputs": {"vae_name": "qwen_image_vae.safetensors"}},
        "12": {"class_type": "LatentUnsqueeze", "inputs": {"samples": ["3", 0]}}
    }
    
    headers = {"Authorization": f"Bearer {SERVICES['comfy']['token']}"}
    try:
        start = time.time()
        resp = requests.post(f"{SERVICES['comfy']['url']}/prompt", json={"prompt": workflow}, headers=headers)
        if resp.status_code != 200: return None, 0
        
        prompt_id = resp.json()['prompt_id']
        while True:
            hist = requests.get(f"{SERVICES['comfy']['url']}/history/{prompt_id}", headers=headers)
            if hist.status_code == 200 and hist.json(): break
            time.sleep(1)
            
        duration = time.time() - start
        history = hist.json()[prompt_id]
        img_info = history['outputs']['9']['images'][0]
        img_resp = requests.get(f"{SERVICES['comfy']['url']}/view?filename={img_info['filename']}&subfolder={img_info['subfolder']}&type={img_info['type']}", headers=headers)
        return Image.open(BytesIO(img_resp.content)), duration
    except Exception as e:
        print(f"Comfy Error: {e}")
    return None, 0

✅ Configuration chargée


In [ ]:
# 3. Comparaison en Action
prompt = "A cute robot playing chess in a park, sunlight, detailed"

print("🎨 Génération Forge (SDXL Turbo)...")
img_forge, time_forge = generate_forge(prompt)

print("🎨 Génération Z-Image (Lumina-2)...")
img_z, time_z = generate_z_image(prompt)

# Affichage
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

if img_forge:
    axes[0].imshow(img_forge)
    axes[0].set_title(f"SDXL Turbo ({time_forge:.2f}s)\n512x512")
else:
    axes[0].text(0.5, 0.5, "Forge Offline", ha='center')
axes[0].axis("off")

if img_z:
    axes[1].imshow(img_z)
    axes[1].set_title(f"Z-Image ({time_z:.2f}s)\n1024x1024")
else:
    axes[1].text(0.5, 0.5, "Z-Image Error", ha='center')
axes[1].axis("off")

plt.show()

## Analyse
- **Vitesse :** SDXL Turbo devrait être quasi instantané (<1s sur GPU récent), tandis que Z-Image prend plusieurs secondes.
- **Résolution :** SDXL Turbo est optimisé pour 512px, Z-Image brille en 1024px.
- **Qualité :** Observez les détails et la cohérence. Z-Image (Lumina-2) a une meilleure compréhension du prompt complexe.